In [38]:
from typing import Dict, List, Tuple, Union, Any, Optional

from functools import partial
import itertools as it

import pandas as pd
import numpy as np
import datetime as dt
import os, sys

from pathlib import Path

import torch
import ray
from ray import train, tune

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from tqdm import tqdm
from wrapt_timeout_decorator import timeout

from src.datasets import daocensus_text
from src.utils import Timer
from src.utils import notebooks as nbutils
from src import paths

sns.set_theme()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Parameters

In [39]:
JOBLIB_CACHE_DIR = './.cache/11_pln-tune'

TRANSFORMER_MODEL = "all-mpnet-base-v2"
ORG_NAME = 'Decentraland'
ORG_PLATFORM: str = 'snapshot'
USE_ORG_NAMES: bool = True

# Evaluation
# N_SPLITS: int = 10
# SKIP_SPLIT: int = 0
K_RECOMMENDATIONS: List[int] = [5, 10]
SPLITS_FREQ: str = 'W-THU' # split weekly
SPLITS_NORMALIZE = True # Wether or not to move everything to 00:00
LAST_SPLITS = 10 # Use just last 10 splits
CUTOFF_DATE_STR: str = '2023-07-29'

MDF_FILE: str = None

# k for K Nearest Neighbors
# KNN_K: int = 1

# In the PLN knn model, first if trains with one set of hparams
# If it takes more than this number of seconds to do just one
# It explores a much simple space
PLN_KNN_TEST_ALL_CUTOFF_SECONDS: int = 60

GRAPHS_METRIC = 'map@10'

In [41]:
CUTOFF_DATE = dt.datetime.fromisoformat(CUTOFF_DATE_STR) if CUTOFF_DATE_STR else None

print('CUTOFF_DATE', CUTOFF_DATE, type(CUTOFF_DATE))
# Metrics Dataframe
MDF_FILE = Path(MDF_FILE) if MDF_FILE else paths.pln_mdf(ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, cutoff_date=CUTOFF_DATE)
print(MDF_FILE)

CUTOFF_DATE 2020-11-06 00:00:00 <class 'datetime.datetime'>
data/pln/mdf-MetaCartel - MetaCartel Ventures-W-THU-normalize-cutoff_date=2020-11-06 00:00:00.pkl


In [42]:
# dfptext = pd.read_csv('./snapshot_proposals.csv')[['proposal_id', 'title', 'description', 'start', 'end']]
# dfv, dfp = daocensus.get("./data/daos-census", ORG_NAME, 'snapshot')
# dfv['voter'] = dfv['voter'].astype('str')
# dfp = dfp.merge(dfptext, how='left', left_on='platform_proposal', right_on='proposal_id')
# dfp[['start', 'end']] = dfp[['start', 'end']].astype('datetime64')

print(ORG_NAME, ORG_PLATFORM, USE_ORG_NAMES, CUTOFF_DATE)
dfv, dfp = daocensus_text.get("./data/daos-census-text", ORG_NAME, ORG_PLATFORM, use_org_names=USE_ORG_NAMES, cutoff_date=CUTOFF_DATE)
dfv[['voter', 'proposal']] = dfv[['voter', 'proposal']].astype(str)
dfp[['id']] = dfp[['id']].astype(str)
dfp = dfp.set_index('id')
print(dfv.info())
print(dfp.info())

MetaCartel - MetaCartel Ventures ['daohaus', 'snapshot'] True 2020-11-06 00:00:00


,platform,name,id,proposal,deployment,platform_vote,voter,date,choice,weight
0,daohaus,MetaCartel - MetaCartel Ventures,a7a47c63-4c4d-58ae-88ee-f6ce6946d088,b23fab43-3b08-5638-b13d-0e286cadb04b,b5c669ac-488c-5b91-82cc-e98add7af0b7,0x0372f3696fa7dc99801f435fd6737e57818239f2-mem...,0x0eabffd8ce94ab2387fc44ba32642af0c58af433,2020-04-01 03:11:46,1,NaN
1,daohaus,MetaCartel - MetaCartel Ventures,f1a74998-e2f3-55f0-964e-f3a8aa56c5c5,18f8c9fd-9796-5802-ab78-b449c65ffb50,b5c669ac-488c-5b91-82cc-e98add7af0b7,0x0372f3696fa7dc99801f435fd6737e57818239f2-mem...,0x0eabffd8ce94ab2387fc44ba32642af0c58af433,2020-06-17 03:08:43,1,NaN
2,daohaus,MetaCartel - MetaCartel Ventures,ae6f03c5-adb5-50d5-849b-761a928ec3f6,2a062f74-c7cb-5625-b8ba-537b23ae5318,b5c669ac-488c-5b91-82cc-e98add7af0b7,0x0372f3696fa7dc99801f435fd6737e57818239f2-mem...,0x0eabffd8ce94ab2387fc44ba32642af0c58af433,2019-07-25 20:06:53,1,NaN
3,daohaus,MetaCartel - MetaCartel Ventures,c00e1243-5304-521c-8c98-eae0c80619b4,3eff52ec-93aa-5c41-9ff2-3a18d278d138,b5c669ac-488c-5b91-82cc-e98add7af0b7,0x0372f3696fa7dc99801f435fd6737e57818239f2-mem...,0x0eabffd8ce94ab2387fc44ba32642af0c58af433,2019-07-30 18:13:46,1,NaN
4,daohaus,MetaCartel - MetaCartel Ventures,98963761-315a-530b-9210-aaae317182b1,b21a24a0-1a16-55a1-8c0e-c97ed7cf8e69,b5c669ac-488c-5b91-82cc-e98add7af0b7,0x0372f3696fa7dc99801f435fd6737e57818239f2-mem...,0x0eabffd8ce94ab2387fc44ba32642af0c58af433,2019-08-06 18:25:25,1,NaN
...,...,...,...,...,...,...,...,...,...,...
1100,daohaus,MetaCartel - MetaCartel Ventures,b6e5b58c-3f16-51ec-b4a7-4f7be375709a,ed418dda-7d6f-5e96-abfc-be7e834082b6,899cc74b-f242-5fd2-a270-763436927977,0x4570b4faf71e23942b8b9f934b47ccedf7540162-mem...,0xfdc6a17917115498a3ed417d4a89224685f26ea3,2020-10-13 16:30:12,1,769.0
1101,daohaus,MetaCartel - MetaCartel Ventures,646da2cf-29ba-5257-9396-0dba4ed0310f,b1ee7c7d-25d9-5fe4-90d3-7cac48beadda,899cc74b-f242-5fd2-a270-763436927977,0x4570b4faf71e23942b8b9f934b47ccedf7540162-mem...,0xfdc6a17917115498a3ed417d4a89224685f26ea3,2020-10-20 03:11:35,1,769.0
1102,daohaus,MetaCartel - MetaCartel Ventures,f9760482-0a78-55d2-8ac0-750a29b7670a,ea49447a-f0a2-5bba-8ad5-45df68670c67,899cc74b-f242-5fd2-a270-763436927977,0x4570b4faf71e23942b8b9f934b47ccedf7540162-mem...,0xfdc6a17917115498a3ed417d4a89224685f26ea3,2020-10-20 03:11:52,1,769.0
1103,daohaus,MetaCartel - MetaCartel Ventures,44f10cb1-5a02-5602-b979-d493de67b502,0dc04c0d-be1b-5b91-8454-d465ac413f96,899cc74b-f242-5fd2-a270-763436927977,0x4570b4faf71e23942b8b9f934b47ccedf7540162-mem...,0xfdc6a17917115498a3ed417d4a89224685f26ea3,2020-10-20 03:11:52,1,769.0


Warning, removing 1105 votes without proposal
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1105 entries, 0 to 1104
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   platform       1105 non-null   object        
 1   name           1105 non-null   object        
 2   id             1105 non-null   object        
 3   proposal       1105 non-null   object        
 4   deployment     1105 non-null   object        
 5   platform_vote  1105 non-null   object        
 6   voter          1105 non-null   object        
 7   date           1105 non-null   datetime64[ns]
 8   choice         1105 non-null   object        
 9   weight         686 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 95.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 373 entries, 2a062f74-c7cb-5625-b8ba-537b23ae5318 to 411dc063-5cd7-5de3-ac9a-5c9dba3f721a
Data columns (total 12 

## Getting the embeddings of each proposal

In [43]:
from sentence_transformers import SentenceTransformer
import pickle

str(model)
```
SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)
```

hash(str(model)) = 1078797550815066609

In [44]:
model = SentenceTransformer(TRANSFORMER_MODEL)

In [45]:
from src.models.nlp import get_embeddings_from_cache

model = SentenceTransformer(TRANSFORMER_MODEL)
print(model)
embeddings = get_embeddings_from_cache(dfp, model)

# We have to delete the model because it complains about forking
del model
# ids = embeddings.index
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)
All embeddings are already calculated


In [46]:
embeddings_sum = embeddings.sum(axis=0)
print(embeddings.shape, embeddings_sum.shape)
embeddings = embeddings.apply(lambda x: x / embeddings_sum)
embeddings

(373,) (768,)


id
2a062f74-c7cb-5625-b8ba-537b23ae5318    [0.0053619323, 0.0029296754, -0.0026650855, 0....
9013374b-028a-5a7b-8b6a-95079c23f99e    [0.0048854207, 0.007195598, 0.0025023418, 0.00...
e473d69a-04e8-5e28-b609-1a55b698351c    [0.0038528352, 0.011546231, 0.0039255084, 0.00...
5471b1ef-9492-57d1-a3ad-7894cbfc18cd    [0.01405283, 0.010552851, -0.0032659138, 0.016...
57f4ceee-4abb-5d0d-bebf-24ae552f3656    [0.0056888545, 0.0049816775, 0.007084623, 0.00...
                                                              ...                        
3362b375-573e-598e-978a-b7d7d3d0f60f    [0.0025422564, -0.0010741148, -0.026476627, 0....
fd269cf9-de0b-576e-9c52-e0236f92e243    [0.0073005287, -0.0048108012, -0.0024054106, 0...
c93d40d0-1671-549a-973c-fe8a9b576406    [0.0013113831, -0.0027935347, -0.004309634, 0....
49c5fff0-672f-5332-a486-2fa2bfa78aef    [0.0024661398, 0.00064929033, 0.01145855, -0.0...
411dc063-5cd7-5de3-ac9a-5c9dba3f721a    [-0.0024823996, 0.0022096946, 0.018924, 0.0032...
Length:

In [47]:
dfp['embeddings'] = embeddings

## Getting the embeddings of each user

The embeddings of each user are the embeddings of each proposal the user has voted in

In [48]:
# dfp['nbidx'] = dfp.index
# _dfv = dfv.merge(dfp[['id', 'nbidx']], how='left', left_on='proposal', right_on='id')
# print('dfp:', dfp.shape, 'dfv:', dfv.shape, '_dfv:', _dfv.shape)
# votes_embeddings = embeddings[_dfv['nbidx']]
votes_embeddings = embeddings.loc[dfv['proposal']]
print('dfp:', dfp.shape, 'dfv:', dfv.shape)
votes_embeddings.shape

dfp: (373, 13) dfv: (1105, 10)


(1105,)

In [49]:
# observed = True, because voter is categorical and might have no votes really
voter_embeddings = dfv.groupby('voter', observed=True).apply(lambda x: votes_embeddings[x.index].sum(axis=0))
voter_embeddings = voter_embeddings.apply(lambda x: x/np.linalg.norm(x))
# msk = (np.linalg.norm(np.stack(voter_embeddings.to_numpy()), axis=1) > 0.9)
voter_embeddings

voter
0x06601571aa9d3e8f5f7cdd5b993192618964bab5    [0.0063951383, 0.0036070677, 0.0062679467, 0.0...
0x0eabffd8ce94ab2387fc44ba32642af0c58af433    [0.004782884, 0.005478198, 0.0052445643, 0.003...
0x11df180d9bdbe90fa51898b3d59959549d589426    [0.0027508016, -0.0017801926, -0.000534651, 0....
0x1289f94bcc60ed9f894ab9d5a54c21b3d4b3f2da    [0.0010701924, 0.0042284853, 0.0041816407, 0.0...
0x1bef65e44adc3fb394cecca4750a1666310e6e2e    [0.004170979, 0.0020558357, 0.005003567, 0.002...
                                                                    ...                        
0xf7f189082878846c11a94ddac51c41afc7a7c772    [0.0034193979, 0.0036491407, 0.0002554522, 0.0...
0xf8a685702347377e396486967399f33c6c0c0a29    [0.0026930743, 0.0026091714, 0.0026843136, 0.0...
0xfafd1d1d502ba454a5fb3d89ede92a45bf8d8d8f    [0.0026225534, 0.0008200375, 0.0076404954, 0.0...
0xfdc6a17917115498a3ed417d4a89224685f26ea3    [0.004279718, 0.0014618227, 0.005112526, 0.001...
0xffd1ac3e8818adcbe5c597ea076e8d32

In [50]:
dfv.groupby('voter', observed=True).size().sort_values()

voter
0x8f942eced007bd3976927b7958b50df126feecb5      1
0x9492510bbcb93b6992d8b7bb67888558e12dcac4      1
0x57db4c6e862e4144ff48b67732d2ccb5af9de14c      1
0xc6b0a4c5ba85d082ecd4fb05fbf63eb92ac1083a      1
0xb6ef7fffd7ae5718287d04cb6a077197e3240fc0      1
                                             ... 
0xbaf6e57a3940898fd21076b139d4ab231dcbbc5f     50
0xd26a3f686d43f2a62ba9eae2ff77e9f516d945b9     53
0xe2a82cdccbfa6ebf9817b0c4aed45264bd41fbec     54
0xc864c34b7313e55bc832b857d8de55ff0f0d0b28    106
0x865c2f85c9fea1c6ac7f53de07554d68cb92ed88    130
Length: 96, dtype: int64

## Making recommendations with cosine similarity

Because they are normalized, our ranking score can be just the dot product of a user and a proposal

In [51]:
embeddings_np = np.vstack(embeddings)

In [52]:
# user_emb = voter_embeddings['0x613e052555ac74ff6af0fc64e40e8035c1e9dcf8']
# scores = embeddings_np @ user_emb
# scores.argsort()

In [53]:
def recommend_emb(emb, k, t = None):
    """
    Args:
        emb: The embedding of the item/user to recommend items to
        k (int): The number of items to recommend
        t (timestamp): Simulate that the recommendation is made at time `t`
            i.e: Filter out already closed proposals or not yet open
    """
    # 1D n_proposals
    scores = embeddings_np @ emb

    if t:
        msk = (dfp['start'] < t) & (t < dfp['end'])
        scores[msk] = -np.inf
    
    best = (-scores).argsort()

    dfbest = dfp.iloc[best[:k]].copy()
    dfbest['score'] = scores[best[:k]]
    return dfbest

In [54]:
def recommend_user(voter: str, k: int = 5, t: Optional[np.datetime64] = None):
    # 1D emb_size
    user_emb = voter_embeddings[voter]
    return recommend_emb(user_emb, k, t)

# recommend_user('0x613e052555ac74ff6af0fc64e40e8035c1e9dcf8')[['title', 'description', 'score']]

In [55]:
def recommend_proposal(proposal: str, k: int = 5, t: Optional[np.datetime64] = None):
    # 1D emb_size
    prop_emb = dfp['embeddings'][proposal]
    return recommend_emb(prop_emb, k, t)

# display(dfp.loc['433b7e43-77b0-5ea1-bac3-0c3071363a56'])
# recommend_proposal('433b7e43-77b0-5ea1-bac3-0c3071363a56')

### Evaluation

First, we will evaluate if the user recommendations are any good using common off-line metrics.

In [56]:
assert not any(voter_embeddings.isna()), "Hay embeddings NA"

In [57]:
np.vstack(voter_embeddings)

array([[ 0.00639514,  0.00360707,  0.00626795, ...,  0.02001015,
         0.00439718,  0.00071826],
       [ 0.00478288,  0.0054782 ,  0.00524456, ..., -0.00082641,
         0.00288238,  0.00147902],
       [ 0.0027508 , -0.00178019, -0.00053465, ..., -0.00303268,
         0.00219137,  0.00056394],
       ...,
       [ 0.00262255,  0.00082004,  0.0076405 , ..., -0.02108281,
         0.00142444,  0.00405409],
       [ 0.00427972,  0.00146182,  0.00511253, ..., -0.00833723,
         0.00424085,  0.00311396],
       [ 0.00141789,  0.00317247,  0.00115551, ...,  0.00176599,
         0.00076862, -0.00125176]], dtype=float32)

In [58]:
def recommendNLPNaive(train: pd.DataFrame, users, proposals, k: int = 5, remove_training: bool = True, min_score: float = 0):  
    # Create voter embeddings (filtering that they are in train)
    # TODO: Add some kind of discount rate over time
    # TODO: DELETEME
    aux = dfv[dfv['voter'].isin(users) & dfv['proposal'].isin(train['itemID'])]
    assert not aux.empty, "Aux is empty"
    not_in_train = set(users).difference(dfv[dfv['proposal'].isin(train['itemID'])]['voter'])
    # print('aux.shape:', aux.shape)
    # print('not in train (in):', len(not_in_train))

    voter_embeddings = aux.groupby('voter', observed=True).apply(lambda x: votes_embeddings[x.index].sum(axis=0))
    voter_embeddings = voter_embeddings.apply(lambda x: x/np.linalg.norm(x))
    assert not voter_embeddings.empty, "Voter embeddings is empty!"
    # np_voter_embeddings = np.stack(voter_embeddings.to_numpy())
    np_voter_embeddings = np.vstack(voter_embeddings)
    
    # Remove score from proposals not in `proposals`
    msk = embeddings.index.isin(proposals)
    tr_embeddings = np.vstack(embeddings).copy()
    tr_embeddings[~msk] = 0
    
    scores = np_voter_embeddings @ tr_embeddings.T
    
    # Remove score from proposals already voted by user
    if remove_training:
        trainu = train[train['userID'].isin(users)]
        itemID2idx = pd.Series(data=np.arange(len(dfp)), index=dfp.index.astype(str))
        voterID2idx = pd.Series(data=np.arange(len(voter_embeddings)), index=voter_embeddings.index)
        scores[voterID2idx[trainu['userID']], itemID2idx[trainu['itemID']]] = -np.inf
        
    best = (-scores).argsort(axis=1)
    topk = best[:, :k]
    
    # create df with columns
    # userID, itemID, prediction
    uid = np.repeat(np.arange(np_voter_embeddings.shape[0]), k)
    iid = topk.flatten()
    
    # transform int to id
    df = pd.DataFrame({
        'userID': voter_embeddings.index[uid],
        'itemID': dfp.index[iid].astype(str),
        # 'prediction': 1,
        'prediction': scores[uid, iid],
    })
    return df[df['prediction'] > min_score].reset_index(drop=True)

# train = train_filtered
# users = test_filtered['userID'].unique()
# proposals = test_filtered['itemID'].unique()
# k = 3
# remove_training = True

In [59]:
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from src.datasets import to_microsoft
from src.model_selection import timeIntervalSplit, timeFreqSplitCurrent

In [60]:
def check_train_recs(df, filter_by_df, filter_by_cols):
    return any(df.set_index(filter_by_cols).index.isin(
        filter_by_df.set_index(filter_by_cols).index
    ))

### Evaluation in multiple months

In [61]:
df = to_microsoft(dfv)
folds = list(timeFreqSplitCurrent(df, SPLITS_FREQ, dfp.reset_index(), return_open=True, remove_not_in_train_col='userID'))[-LAST_SPLITS:]
metrics_f = { 'precision':precision_at_k, 'ndcg': ndcg_at_k, 'ndcg': ndcg_at_k, 'map': map_at_k, } # 'recall': recall_at_k, 

def testHParamsSimilarity(fold, window_size):
    metrics = dict()

    train, test, t, open_proposals = folds[fold]
    offset = pd.tseries.frequencies.to_offset(window_size)

    train_filtered = train[train['timestamp'] > (t - offset)]
    not_in_train = set(test['userID']).difference(train['userID'])
    # print('not in train:', len(not_in_train))
    # print('not in train filtered:', len(not_in_train))
    for k_recs in K_RECOMMENDATIONS:
        with Timer() as t:
            recs = recommendNLPNaive(train_filtered, test['userID'].unique(), open_proposals, k_recs)

        metrics[f'time@{k_recs}'] = t.time
        for m, f in metrics_f.items():
            metrics[m+f'@{k_recs}'] = f(test, recs, k=k_recs)

    return metrics

testHParamsSimilarity(0, '14d')
# testHParamsSimilarity(1, '2W')

{'time@5': 0.0043741557747125626,
 'precision@5': 0.05,
 'ndcg@5': 0.25,
 'map@5': 0.25,
 'time@10': 0.004226870834827423,
 'precision@10': 0.05,
 'ndcg@10': 0.29179011374155056,
 'map@10': 0.2638888888888889}

In [62]:
from sklearn.model_selection import ParameterGrid

param_grid = ParameterGrid({
    'fold': range(len(folds)),
    'window_size': ['7d', '14d', '30d', '90d', '180d', '360d', '10Y'],
})

results = list()
for p in tqdm(param_grid):
    results.append(testHParamsSimilarity(**p) | p)

mdf = pd.DataFrame(results).set_index(['fold', 'window_size'])
mdf

100%|███████████████████████████████████████████████████| 70/70 [00:04<00:00, 17.49it/s]


time@5  precision@5    ndcg@5     map@5   time@10  \
fold window_size                                                        
0    7d           0.004468         0.10  0.300527  0.270833  0.004194   
     14d          0.004185         0.05  0.250000  0.250000  0.004083   
     30d          0.004211         0.10  0.324020  0.291667  0.004117   
     90d          0.004308         0.05  0.096713  0.050000  0.004251   
     180d         0.004422         0.05  0.096713  0.050000  0.004349   
...                    ...          ...       ...       ...       ...   
9    30d          0.004188         0.24  0.887349  0.855556  0.004172   
     90d          0.004313         0.24  0.887349  0.855556  0.004304   
     180d         0.004458         0.16  0.487349  0.455556  0.004450   
     360d         0.004598         0.16  0.487349  0.455556  0.004462   
     10Y          0.004555         0.16  0.487349  0.455556  0.004517   

                  precision@10   ndcg@10    map@10  
fold window_size                                    
0    7d                  0.075  0.342317  0.284722  
     14d                 0.050  0.291790  0.263889  
     30d                 0.075  0.359337  0.310185  
     90d                 0.025  0.096713  0.050000  
     180d                0.025  0.096713  0.050000  
...                        ...       ...       ...  
9    30d                 0.120  0.887349  0.855556  
     90d                 0.120  0.887349  0.855556  
     180d                0.100  0.558591  0.488889  
     360d                0.100  0.558591  0.488889  
     10Y                 0.100  0.558591  0.488889  

[70 rows x 8 columns]

In [ ]:
sns.lineplot(mdf, x='fold', y='precision@5', hue='window_size')

In [ ]:
mdf.groupby('window_size').describe().drop(columns=['time@10', 'ndcg@10']).T

In [ ]:
from src import baseline

# bdf = pd.read_csv(paths.baseline_mp_freq(ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE))[-LAST_SPLITS:]
bdf = baseline.read_metrics_baseline(ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, cutoff_date=CUTOFF_DATE)[-LAST_SPLITS:]
assert bdf.index.max() < CUTOFF_DATE if CUTOFF_DATE else True
bdf['fold'] = np.arange(10)
bdf.describe()

In [ ]:
# pdf = pd.read_csv(paths.perfect_mp_freq(ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE))[-LAST_SPLITS:]
pdf = baseline.read_metrics_perfect(ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, cutoff_date=CUTOFF_DATE)[-LAST_SPLITS:]
pdf['fold'] = np.arange(10)
pdf.describe()

In [ ]:
_METRICS = ['map@5', 'map@10']
_METRIC = _METRICS[-1]

ax = sns.lineplot(mdf, x='fold', y=_METRIC, hue='window_size')
ax = sns.lineplot(bdf, x='fold', y=_METRIC, color='red', zorder=0)
ax.lines[-1].set_linestyle('--')
ax.text(9, 0.19, "MP Baseline", ha='right', color=ax.lines[-1].get_color())

# ax = sns.lineplot(pdf, x='fold', y=_METRIC, color='blue')
# ax.lines[-1].set_linestyle(':')

ax.set_ylim((0,None));
nbutils.download_matplotlib(ax.figure, f'11_cosine_results_results-lines_{SPLITS_FREQ}_normalize={SPLITS_NORMALIZE}.pdf')

In [ ]:
# _METRIC = 'precision@5'

topline = pdf[_METRIC].mean()

_toplot = mdf[_METRICS].melt(value_vars=_METRICS, var_name='metric', ignore_index=False)

sns.set_theme()
ax = sns.barplot(_toplot, x='window_size', y='value', hue='metric')
ax.set_xlabel("Window size")

baseline = bdf[_METRICS[0]].mean()
ax.axhline(baseline, ls='--', color='blue', zorder=0)
baseline = bdf[_METRICS[1]].mean()
ax.axhline(baseline, ls='--', color='orange', zorder=0)
# ax.text(0, baseline-0.005, "map@5", color='black', ha='left', va='top', fontsize='large')

if not _METRIC.startswith('map@'):
    ax.axhline(topline, ls=':', color='blue')
    ax.text(0, topline-0.005, "perfect", color='blue', ha='left', va='top')

ax2 = ax.twinx()
time_color = 'green'
sns.lineplot(mdf, x='window_size', y='time@10', color=time_color, ax=ax2, markers=True, dashes=(4,2), errorbar=None)
ax2.set_ylabel('time (s)', color=time_color)
ax2.tick_params(axis='y', color=time_color)
ax2.spines['right'].set_color(time_color)
[t.set_color(time_color) for t in ax2.yaxis.get_ticklabels()]
ax2.grid(False)
ax2.set_ylim((0,.10))

ax.legend(loc=(0.02,.2))

print(type(plt.figure()))
nbutils.download_matplotlib(ax.figure, f'11_cosine_results_window-size_{SPLITS_FREQ}_normalize={SPLITS_NORMALIZE}.pdf')

# Second iteration: PLN with kNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# itemID2idx = pd.Series(data=np.arange(len(dfp)), index=dfp.index.astype(str))
# itemID2idx

In [ ]:
def filter_proposals_in_train(train, user, proposals):
    msk = train['userID'] == user
    proposals_aready_voted_by_user = train[msk]['itemID'].unique()
    return np.setdiff1d(proposals, proposals_aready_voted_by_user)

In [ ]:
from joblib import Memory

# memory = Memory(JOBLIB_CACHE_DIR, mmap_mode='c', verbose=0)

# @memory.cache
def recommendUserKNN(train: pd.DataFrame, user, proposals, top_k: Optional[int] = None, knn_k = 3, do_sampling: Optional[bool] = None, remove_train: bool = True, metric=None):
    """
    Given the train data (proposals that might be voted or not by the user), recommend k proposals from proposals to the user
    The train data is a DataFrame in microsoft's format
    
    do_sampling (bool) : default None
        if True, makes sampling so the number of positive and negative samples is the same
        if False, doesn't do it
        if None (default), does it when k > 0, but doesn't do it with Nearest Neighbor
    """
    # train['hasVoted'] = train['userID'] == user
    hasVoted = train['userID'] == user

    # Remove recommendations already in training
    # (aka remove from proposals array)
    if remove_train:
        proposals = filter_proposals_in_train(train, user, proposals)

    assert any(hasVoted), 'The user has not voted in any proposal'
    if knn_k == 0:
        do_sampling = False
        knn_k = 1

    if do_sampling or (do_sampling is None and knn_k > 0):
        # num_samples = train['hasVoted'].value_counts().min() # [True]
        num_samples = hasVoted.value_counts().min() # [True]
        train = train.groupby(hasVoted).apply(lambda x: x.sample(num_samples)).reset_index(drop=True)
        hasVoted = train['userID'] == user

    # known_embeds = embeddings[itemID2idx[train['itemID']]]
    known_embeds = np.vstack(embeddings.loc[train['itemID']])

    assert knn_k >= 0
    knn = KNeighborsClassifier(max(knn_k, 1), metric=metric if metric else 'minkowski')
    knn.fit(known_embeds, hasVoted)# train['hasVoted'])

    predict_embeds = np.vstack(embeddings.loc[proposals])
    aux = knn.predict_proba(predict_embeds)
    p = knn.predict_proba(predict_embeds)[:, 1] # Probabilidad de que sea True

    if top_k:
        best = (1-p).argsort()[:top_k]
        p = p[best]
        proposals = proposals[best]
    
    return pd.DataFrame({
        'userID': user,
        'itemID': proposals,
        'prediction': 1,
        'score': p,
    })

# u = '0xe44782afe398800d7190796018c4b865e32ff806'
# proposals_voted_by_user = test_filtered[test_filtered['userID'] == u]
# print(f'The user {u} voted in {len(proposals_voted_by_user)} proposals')
# p = recommendUserKNN(train, u, test_filtered['itemID'].unique().copy(), 5, do_sampling=True)
# p['correct'] = p['itemID'].isin(proposals_voted_by_user['itemID'])
# p

In [ ]:
from recommenders.datasets.pandas_df_utils import filter_by

def recommendMostVoted(train: pd.DataFrame, user, proposals, top_k: Optional[int] = None, remove_train=True):
    bestVotes = train[train['itemID'].isin(proposals)]['itemID'].value_counts()
    df = pd.DataFrame({
        'userID': user,
        'itemID': bestVotes.index,
        'prediction': 1,
        'score': bestVotes,
    })
    if remove_train:
        df = filter_by(df, train, ['userID', 'itemID'])
    return df.head(top_k).reset_index(drop=True)

# u = '0xe44782afe398800d7190796018c4b865e32ff806'
# proposals_voted_by_user = test_filtered[test_filtered['userID'] == u]
# print(f'The user {u} voted in {len(proposals_voted_by_user)} proposals')
# recommendMostVoted(train_filtered.copy(), u, test_filtered['itemID'].unique(), 5)

In [ ]:
# import multiprocessing as mp
# from multiprocessing import Pool
# from multiprocessing.pool import ThreadPool as Pool

from joblib import Parallel, delayed

def recommendUserKNNwithFallback(train, user, proposals, top_k = None, knn_k: int = 1, remove_train: bool = True, do_sampling = None, metric=None):
    """ If the user has not voted in any proposal in `train`, we fall back to recommending
    the most voted proposal in `proposals`
    """
    if remove_train:
        proposals = filter_proposals_in_train(train, user, proposals)
    if (user == train['userID']).sum() >= max(knn_k, 1):
        df = recommendUserKNN(train, user, proposals, top_k, knn_k, remove_train=False, do_sampling=do_sampling, metric=metric)
        df['scoreType'] = 'knn'
        return df
    else:
        df = recommendMostVoted(train, user, proposals, top_k, remove_train=False)
        df['scoreType'] = 'mv'
        return df

def recommendNLPKNN(train: pd.DataFrame, users, proposals, top_k: int = 5, knn_k: int = 1, do_sampling=None, cpus=-1, chunksize=8, disable_tqdm=False, metric=None):
    # Create voter embeddings
    assert knn_k >= 0

    # return pd.concat(tqdm((
    #     recommendUserKNNwithFallback(train, u, proposals, top_k, knn_k, remove_train=False, do_sampling=do_sampling) for u in users
    # ), desc='users', leave=False, total=len(users)))

    p = partial(
        recommendUserKNNwithFallback, 
        train, 
        proposals=proposals, 
        top_k=top_k,
        knn_k=knn_k,
        do_sampling=do_sampling,
        metric=metric,
    )

    if cpus == 1:
        # return pd.concat(tqdm(
        return pd.concat(
            map(p, users),
            # total=len(users),
            # desc='users',
            # leave=False,
            # disable=disable_tqdm,
        )
        # ))
    else:
        # with Pool(mp.cpu_count() // 4 if cpus==-1 else cpus) as pool:
        #     return pd.concat(
        #         tqdm(
        #             pool.imap(p, users, chunksize=chunksize),
        #             total=len(users),
        #             desc='users',
        #             leave=False,
        #             disable=disable_tqdm,
        #         )
        #     )
        return pd.concat(
            Parallel()(delayed(p)(u) for u in tqdm(users, desc='users', leave=False, disable=disable_tqdm))
        )

with Timer() as timer_one:
    display(recommendNLPKNN(folds[0][0], folds[0][1]['userID'].unique(), folds[0][1]['itemID'].unique(), top_k=1))
timer_one.time

In [ ]:
# print(mp.cpu_count())

### Evaluation

Usaremos el método del codo para ver cual de todos es mejor

In [ ]:
df = to_microsoft(dfv)
folds = list(timeFreqSplitCurrent(df, SPLITS_FREQ, dfp, remove_not_in_train_col='userID', return_open=True))[-LAST_SPLITS:]
metrics_f = { 'precision':precision_at_k, 'ndcg': ndcg_at_k, 'ndcg': ndcg_at_k, 'map': map_at_k, } # 'recall': recall_at_k, 

def testHParamsKNN(fold, window_size, k, metric, cpus=-1, disable_tqdm=True):
    metrics = dict()

    train, test, t, open_proposals = folds[fold]
    offset = pd.tseries.frequencies.to_offset(window_size)
    train_filtered = train[train['timestamp'] > (t - offset)]

    for k_recs in K_RECOMMENDATIONS:
        with Timer() as t:
            recs = recommendNLPKNN(train_filtered, test['userID'].unique(), open_proposals, k_recs, k, disable_tqdm=disable_tqdm, metric=metric, cpus=cpus)

        metrics[f'time@{k_recs}'] = t.time
        metrics[f'knnUsage@{k_recs}'] = (recs['scoreType'] == 'knn').mean()
        for m, f in metrics_f.items():
                metrics[m+f'@{k_recs}'] = f(test, recs, k=k_recs)

    return metrics

# metrics = testHParamsKNN(0, '14d', 0, 'cosine')
# metrics

In [ ]:
from sklearn.model_selection import ParameterGrid
from concurrent.futures import ProcessPoolExecutor as PoolExecutor, as_completed

# Total job timeout: 5 hours
TOTAL_TIMEOUT = 3600 * 5
MAX_WORKERS = 8

param_grid = ParameterGrid({
    'fold': range(len(folds)),
    'window_size': ['7d', '14d', '30d', '90d', '180d', '360d', '10Y'],
    'k': [1,2,3,4,5,6,7,8,9,10,15,20,30,50],
    'metric': ['minkowski', 'cosine'],
})

if timer_one.time > PLN_KNN_TEST_ALL_CUTOFF_SECONDS * 2:
    print("Warning: Using very reduced pgrid (just to keep running the notebook)", file=sys.stderr)
    param_grid = ParameterGrid({
        'fold': range(len(folds)),
        'window_size': ['7d'],
        'k': [1],
        'metric': ['cosine'],
    })
elif timer_one.time > PLN_KNN_TEST_ALL_CUTOFF_SECONDS:
    print("Warning: Using reduced pgrid", file=sys.stderr)
    param_grid = ParameterGrid({
        'fold': range(len(folds)),
        'window_size': ['7d', '14d', '30d'],
        'k': [1,2,3,4,5],
        'metric': ['cosine'],
    })

def _retDict(p):
    # @timeout(60, use_signals=True, exception_message=f"Timeout testing {p}")
    def aux(p):
        return testHParamsKNN(cpus=1, **p)

    return p, aux(p)

done = None
todo_idx = pd.MultiIndex.from_frame(pd.DataFrame(param_grid))

if MDF_FILE.exists():
    done = pd.read_pickle(MDF_FILE)
    todo_idx = todo_idx.reorder_levels(done.index.names).difference(done.index)

todo_recs = [ { n:r for n,r in zip(todo_idx.names, p)} for p in todo_idx ]

if not todo_recs:
    print("All done!")
else:
    with PoolExecutor(max_workers=MAX_WORKERS) as ex:
        fs = [ ex.submit(_retDict, p) for p in todo_recs ]
        ex.shutdown(wait=False)
    
        for f in tqdm(as_completed(fs, timeout=TOTAL_TIMEOUT), total=len(fs)):
            p, res = f.result()
            if done is None or done.empty:
                done = pd.DataFrame(res, index=pd.MultiIndex.from_tuples([p.values()], names=p.keys()))
            else:
                done.loc[tuple(p.values())] = res
                done.to_pickle(MDF_FILE)

In [ ]:
KNN_ANALYZE_METRIC = 'map@10'

In [ ]:
mdf = pd.read_pickle(MDF_FILE)
mdf_mean = mdf.groupby(['k', 'window_size', 'metric']).mean()
mdf_mean.sort_values(KNN_ANALYZE_METRIC)

### Plotting

In [ ]:
cols = [KNN_ANALYZE_METRIC, 'map@5', 'time@10']

(mdf
 .groupby(['window_size', 'k', 'metric'])[cols]
 .describe()
 .sort_values((KNN_ANALYZE_METRIC, 'mean'), ascending=False)
 .loc[:, (slice(None), ['mean', 'std'])]
 .head(10)
)

In [ ]:
import matplotlib.ticker as mtick

ax = (mdf.groupby('k')[['knnUsage@5', 'knnUsage@10']].mean()*100).plot(style='.-')
ax.set_ylim((0, 100))
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
nbutils.download_matplotlib(ax.figure, f'11_knn_usage_k_{SPLITS_FREQ}_normalize={SPLITS_NORMALIZE}.pdf')

In [ ]:
order = param_grid.param_grid[0]['window_size']
_toplot = (mdf.groupby('window_size')[['knnUsage@5', 'knnUsage@10']].mean()*100).loc[order]

ax = _toplot.plot(style='.-')
ax.set_ylim((0, 100))
ax.set_xlabel('Tamaño de ventana')
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
nbutils.download_matplotlib(ax.figure, f'11_knn_usage_window_size_{SPLITS_FREQ}_normalize={SPLITS_NORMALIZE}.pdf')

In [ ]:
_toplot = mdf_mean.reset_index().copy()
_toplot['days'] = pd.to_timedelta(_toplot['window_size']).dt.days
_toplot = _toplot.sort_values(['k', 'days'])
# _toplot['color'] = _toplot['window_size'] + ',' + _toplot['metric']

fig = px.line(
    _toplot, 
    x='k', 
    y=KNN_ANALYZE_METRIC, 
    color='window_size', 
    markers=True, 
    line_dash='metric', 
    hover_data='time@10',
    # color_discrete_sequence=px.colors.sequential.Turbo,
)
display(nbutils.download_plotly(fig, f'11_knn_results_k-size_{SPLITS_FREQ}_normalize={SPLITS_NORMALIZE}.pdf'))
fig

In [ ]:
baseline = baseline = bdf[KNN_ANALYZE_METRIC].mean()

ax = sns.lineplot(_toplot, 
    x='k', 
    y=KNN_ANALYZE_METRIC, 
    hue='window_size', 
    # markers=True, 
    style='metric', 
    # hover_data='time@10',
)
ax.axhline(baseline, linestyle=':')
ax.text(0, baseline, "baseline", 
        color=ax.lines[-1].get_color(),
        ha='left', 
        va='bottom',
        fontsize='large',
)
# ax.set_xlabel('log')
# ax.set_ylim((0,0.7))
nbutils.download_matplotlib(ax.figure, f'11_knn_results_all_{SPLITS_FREQ}_normalize={SPLITS_NORMALIZE}.pdf')

In [ ]:
baseline = baseline = bdf[KNN_ANALYZE_METRIC].mean()

ax = sns.lineplot(_toplot, 
    x='k', 
    y='time@10', 
    hue='window_size', 
    # markers=True, 
    style='metric', 
    # hover_data='time@10',
)
# ax.set_xscale('log')
ax.set_ylabel('time (s)')
nbutils.download_matplotlib(ax.figure, f'11_knn_results_time_{SPLITS_FREQ}_normalize={SPLITS_NORMALIZE}.pdf')

In [ ]:
_toplot = mdf.groupby(['window_size', 'metric']).mean().reset_index()
_toplot['days'] = pd.to_timedelta(_toplot['window_size']).dt.days
_toplot = _toplot.sort_values('days')

px.bar(
    _toplot,
    x='window_size',
    y=KNN_ANALYZE_METRIC,
    color='metric',
    hover_data='time@10',
    # pattern_shape='metric',
    barmode='group',
)